In [ ]:
import numpy as np
from hmmlearn import hmm
import matplotlib.pyplot as plt

# create a discrete HMM for the unfair casino with 2 states (fair and unfair dice)
unfair_casino = hmm.CategoricalHMM(n_components=2)

# number of features per state
unfair_casino.n_features_ = 6

# probability to start in a given dice
unfair_casino.startprob_ = np.array([1.0, 0.0])

# transition matrix (A)
unfair_casino.transmat_ = np.array([[0.95, 0.05], [0.2, 0.8]])

# emission probability
unfair_casino.emissionprob_ = np.array(
    [
        1.0 / 6,
        1.0 / 6,
        1.0 / 6,
        1.0 / 6,
        1.0 / 6,
        1.0 / 6,
        0.01,
        0.01,
        0.01,
        0.01,
        0.01,
        0.95,
    ]
).reshape((2, 6))
# evaluate a sequence  (emission 5 is the loaded 6)
example = [
    0,
    5,
    1,
    2,
    4,
    3,
    5,
    5,
    5,
    5,
    5,
    1,
    2,
    3,
    0,
    4,
    0,
    1,
    3,
    2,
    5,
    1,
    2,
    4,
    5,
    5,
    5,
    4,
    5,
    1,
    2,
    3,
    0,
]

In [ ]:
# Viterbi algorithm on the model and emission sequence as a vertical vector
example = np.array(example).reshape((33, 1))
unfair_casino.decode(example)
# result is a likelihood and a sequence of states - 0 =fair, 1=loaded

In [ ]:
# visualizing reconstructed states and emissions
plt.plot(example, "r*", label="thrown values")
plt.plot(unfair_casino.decode(example)[1], "b-", label="reconstructed states")
plt.legend(loc="upper left")

In [ ]:
unfair_casino.sample(10)

In [ ]:
# sample data from model

res = unfair_casino.sample(100)
example2 = res[0]

# run the Viterbi algorithm and plot

res2 = unfair_casino.decode(example2)

# plot the posterior distribution
plt.figure(figsize=(6, 4), dpi=150)
plt.plot(res[1], "g-", label="states")
plt.plot(res2[1], "b:", label="reconstruction")
plt.plot(example2, "r.", label="thrown values")

plt.legend()
plt.show()

In [ ]:
sample3, res3 = unfair_casino.sample(20000)[0:2]

# initializing a new HMM model
unfair_casino2 = hmm.CategoricalHMM(n_components=2)
unfair_casino2.n_features_ = 6

unfair_casino2.fit(sample3)

print(unfair_casino2.emissionprob_)
print(unfair_casino2.transmat_)
# resulting fit - not very good, however there is something about enrichment of emission 5 in state 0

decoded3 = unfair_casino2.decode(sample3)
decoded3b = unfair_casino.decode(sample3)

# plot the posterior distribution (just the beginning 200)
plt.figure(figsize=(6, 4), dpi=150)
plt.plot(res3[:200], "g-", label="states")
plt.plot(decoded3[1][:200], "b:", label="reconstruction - estimated model")
plt.plot(decoded3b[1][:200], "y:", label="reconstruction - true model")
plt.plot(sample3[:200], "r.", label="thrown values")

plt.legend()
plt.show()
# Usually not a great estimate

In [ ]:
# initializing a new HMM model, this time with some guidance
unfair_casino3 = hmm.CategoricalHMM(n_components=2, init_params="")
unfair_casino3.n_features_ = 2
unfair_casino3.startprob_ = np.array([1.0, 0.0])
unfair_casino3.transmat_ = np.array([[0.9, 0.1], [0.1, 0.9]])
unfair_casino3.emissionprob_ = np.array(
    [1.0 / 6, 1.0 / 6, 1.0 / 6, 1.0 / 6, 1.0 / 6, 1.0 / 6, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5]
).reshape((2, 6))
# e
unfair_casino3.fit(sample3)

print(unfair_casino3.emissionprob_)
print(unfair_casino3.transmat_)

# usually  we get a much better estimate now

decoded4 = unfair_casino3.decode(sample3)

# plot the posterior distribution (actually the first 200 steps of it)
plt.figure(figsize=(6, 4), dpi=150)
plt.plot(res3[:200], "g-", label="states")
plt.plot(decoded4[1][:200], "b:", label="reconstruction")
plt.plot(sample3[:200], "r.", label="thrown values")

plt.legend()
plt.show()